In [1]:
import MindMap
import sys
import os
import json

from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate,LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from openai import OpenAI
from neo4j import GraphDatabase, basic_auth

/mnt/cephfs/home/wangqianyue/miniconda3/envs/ltg/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#数据库连接
uri = "bolt://202.38.247.177:7687"
username = "neo4j"
password = "Wangqianyue1"

driver = GraphDatabase.driver(uri, auth=(username, password), encrypted=False)
session = driver.session()

In [3]:
session.run("MATCH (n) DETACH DELETE n")
#仅在首次连接时使用，跑完就直接关

In [10]:
#块3.4

#set 初始信息

################################

setting='The story is set in a post-apocalyptic world where the world has ended in a cataclysmic event.\n'
   
character="""
Jake Peters: Jake Peters is a lovable and loyal dog who is the furry friend of the group of survivors.\n
Johnathan Prescott: Johnathan Prescott is a strong and handsome young man who is the leader of the group of survivors.\n
Maria Martinez: Maria Martinez is a beautiful and tough young woman who is the second in command of the group of survivors.\n
Ratzinger Kugelman: Ratzinger Kugelman is a creepy and mysterious man who is the enemy of the group of survivors.\n
    """
outline="""
1. The world ends in a cataclysmic event.\n
2. A group of survivors find themselves in a strange new world where they must band together to survive the perils that await them. \n
3. The group of survivors must face the enemy and defeat him in order to survive in this new world. \n
  
    """
title="test4"

##################################


stage_schema = ResponseSchema(name="chapter",
                                description="Was a string that describes the name of the chapter for a specific storyline.")
storyline_schema = ResponseSchema(name="plan",
                                        description="Was a string that describes the outline or the generation plan for a chapter of the specific episodic long story.")
response_schemas = [stage_schema,
                        storyline_schema]

output_parserp = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions_plan = output_parserp.get_format_instructions()
print(format_instructions_plan)



The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"chapter": string  // Was a string that describes the name of the chapter for a specific storyline.
	"plan": string  // Was a string that describes the outline or the generation plan for a chapter of the specific episodic long story.
}
```


In [11]:
#3.5

#设置初始内容
MindMap.set_initial([setting,character,outline],driver,title)
#只跑一次

                  head            relation                               tail
0                world              set in             post-apocalyptic world
1    cataclysmic event              caused                   end of the world
2          Jake Peters                  is            a lovable and loyal dog
3   Johnathan Prescott                  is    a strong and handsome young man
4       Maria Martinez                  is  a beautiful and tough young woman
5   Ratzinger Kugelman                  is        a creepy and mysterious man
6            The world             ends in                  cataclysmic event
7   group of survivors        finds itself                  strange new world
8   group of survivors  must band together                  to survive perils
9   group of survivors           must face                              enemy
10  group of survivors         must defeat                                him
11            survival         required in                      

In [12]:
#3.6

#寻找每段rough storyoutline 或 dynamic outline的相关历史信息

#大段找历史

###############################

#填入块2.6的输出
text="""

In the aftermath of a catastrophic event, our story begins in a desolate wasteland that was once a thriving metropolis. We meet Jake Peters, a sentient dog, and his human companions - the resilient leader Johnathan Prescott and the fierce Maria Martinez. They form a tight-knit group of survivors, scavenging for resources and shelter in this unforgiving new world.

"""

#对应2.6中的chapter 的值
step=1
#?  大段段数？

title="test4"

###########################

history=MindMap.find_relevant_info(text,step,title,driver)

match_kg ['post-apocalyptic world', 'Jake Peters', 'Johnathan Prescott', 'Maria Martinez', 'group of survivors', 'survival', 'to survive perils', 'world', 'The world', 'cataclysmic event', 'end of the world', 'a lovable and loyal dog']
neighbor_list [['Jake Peters', 'is', 'a lovable and loyal dog'], ['Johnathan Prescott', 'is', 'a strong and handsome young man'], ['Maria Martinez', 'is', 'a beautiful and tough young woman'], ['group of survivors', 'finds itself', 'strange new world'], ['group of survivors', 'must band together', 'to survive perils'], ['group of survivors', 'must face', 'enemy'], ['group of survivors', 'must defeat', 'him'], ['survival', 'required in', 'new world'], ['world', 'set in', 'post-apocalyptic world'], ['The world', 'ends in', 'cataclysmic event'], ['cataclysmic event', 'caused', 'end of the world']]
Jake Peters is described as a lovable and loyal dog.
Johnathan Prescott is described as a strong and handsome young man.
Maria Martinez is described as a beautifu

In [7]:
#寻找每段rough storyoutline 或 dynamic outline的相关历史信息

#小段找历史

text="""
Plot development:
- Shannon, now more familiar with the city's dynamics, takes on her newfound role as a journalist seriously.
- She begins investigating a series of mysterious disappearances in the inner city, suspecting there might be a connection to Gary's community.
- Shannon enlists Gary's help to gather information discreetly, gaining his trust further.
- As they dig deeper, they stumble upon a corrupt network involving local authorities and business owners, including Lena's shop.
- Lena, unaware of the sinister activities, becomes a pawn in a larger scheme, putting both her and Gary in danger.
- Shannon faces ethical dilemmas as she balances her commitment to the truth with the potential harm it could cause.

Key events:
1. Shannon choosing the disappearance cases as her next investigation.
2. Shannon confiding in Gary about her suspicions and seeking his assistance.
3. Gary providing crucial insights into the community's secrets.
4. Discovery of the corrupt network implicating Lena's business.
5. Lena unknowingly becoming entangled in the dangerous situation.
6. Shannon grappling with the moral implications of exposing the corruption.
"""
step=2#对应2中的chapter 的值1-15
title="test2"#对应2中的title
######################################


history=MindMap.find_relevant_info(text,step,title,driver)

match_kg ['Shannon Doyle', "Shannon's father", 'Shannon', 'Gary', 'Lena Saunders', 'successful journalist', 'local business owner', 'truth is more than story']
result_path [[], ['Shannon', 'meets', 'Gary'], [], []]
neighbor_list [['Shannon Doyle', 'age', 'early twenties']]
neighbor_list [['Shannon Doyle', 'age', 'early twenties'], ['Shannon Doyle', 'is', 'young woman']]
neighbor_list [["Shannon's father", 'dies', 'unexpectedly']]
neighbor_list [['Shannon', 'learns', 'truth is more than story']]
neighbor_list [['Shannon', 'learns', 'truth is more than story'], ['Shannon', 'understands', 'harsh realities']]
neighbor_list [['Shannon', 'learns', 'truth is more than story'], ['Shannon', 'understands', 'harsh realities'], ['Shannon', 'meets', 'Gary']]
neighbor_list [['Shannon', 'learns', 'truth is more than story'], ['Shannon', 'understands', 'harsh realities'], ['Shannon', 'meets', 'Gary'], ['Shannon', 'discovers', 'reality differs from dream']]
neighbor_list [['Shannon', 'learns', 'truth i

In [18]:
#set_history

#存小段文本

chapter=15
#1-15


text="""
As Diane stood facing Mark in their dimly lit living room, the tension crackling between them was palpable. The air was heavy with unspoken words and unresolved emotions, each one a weight pressing down on both their hearts.\n

Their conversation started tentatively, with Diane's voice barely above a whisper as she asked the question that had been haunting her since that fateful moment of revelation. "Why, Mark? Why did you do it?" Her eyes searched his, desperate for some semblance of an explanation that could make sense of the shattered pieces of her reality.\n

Mark hesitated, his gaze flickering away before returning to meet hers. The pain etched on his face mirrored the anguish in Diane's heart. "I don't know," he admitted, his voice hoarse with emotion. "I was lost, confused... I made a terrible mistake, and I can't undo it."\n

Their words hung in the air, heavy with the weight of betrayal and regret. Diane felt a surge of anger rise within her, threatening to consume her fragile composure. But beneath the anger, there was a flicker of something else - a glimmer of love, buried deep but refusing to be extinguished.\n

As they delved deeper into their shared history, exploring the cracks and fissures that had long been ignored, Diane and Mark found themselves treading a treacherous path through a minefield of hurt and deception. Each revelation cut deeper than the last, laying bare the vulnerabilities that had festered beneath the surface of their marriage.\n

Mark, too, bared his soul, revealing his inner turmoil and the depths of his regrets. His words were a balm to Diane's wounded spirit, offering a sliver of hope amidst the wreckage of their shattered trust.\n

Diane's emotions were a tumultuous sea, roiling with conflicting currents of anger, hurt, and love. She grappled with the daunting prospect of forgiveness, struggling to find a way to reconcile the man she thought she knew with the one who stood before her now.\n

And then, in a moment of quiet clarity, their conversation reached a crescendo, a pivotal moment where the fate of their marriage hung in the balance. The resolution of their discussion was a turning point, a fragile bridge between their past and an uncertain future.\n

As they sat in the fading light of day, Diane and Mark found themselves at a crossroads, each faced with a choice that would shape the course of their lives. In that moment of vulnerability and truth, they discovered the power of redemption and the possibility of a new beginning.\n

Their journey was far from over, but with hearts laid bare and spirits intertwined, Diane and Mark took the first tentative steps towards healing and reconciliation, determined to forge a path forward despite the scars of their past.\n
"""

MindMap.set_history(text,driver,title,step=chapter)

              head                           relation  \
0            Diane                      stands facing   
1          tension                is palpable between   
2              air                      is heavy with   
3    Diane's voice                       barely above   
4            Diane                               asks   
5         question                  had been haunting   
6     Diane's eyes                             search   
7             gaze  flickers away and returns to meet   
8             pain                       is etched on   
9          anguish                         mirrors in   
10            Mark                             admits   
11           voice                     is hoarse with   
12           words                     are heavy with   
13           Diane                              feels   
14           anger                       rises within   
15  something else                     is buried deep   
16            they             